In [1]:
import pandas as pd
import numpy as np
import datetime
import re

In [70]:
df = pd.read_csv('all_permit_v1.csv')

C:\Users\danie\anaconda_3\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (0,2,9,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [71]:
df.head(3)

structure_type            parcel neighborhood                        owner  \
0            NaN  0048s00284000000          NaN  pfohl donald e & lucy g (w)   
1            NaN  0006r00206000000          NaN            p7 investments lp   
2            NaN  0006r00206000000          NaN                dugan timothy   

                issued                                   work description  \
0  2012-01-30 00:00:00  interior fitout for yoga studio\n-pittsburgh b...   
1  2012-01-04 00:00:00  new 3 story single family dwelling with 2 car ...   
2  2012-01-04 00:00:00  new 3 story single family dwelling with 2 car ...   

       permit           contractor                                address  \
0  11-b-03000  nh construction llc          3418 penn ave, pittsburgh, pa   
1  11-b-03019  heartland homes inc      435 sweetbriar st, pittsburgh, pa   
2  11-b-03021  heartland homes inc  437 sweetbriar st 435, pittsburgh, pa   

  work_type  ward  id      cost  year    month source  mn  
0       NaN   6.0 NaN  200000.0  2012  january    xls   1  
1       NaN  19.0 NaN  100000.0  2012  january    xls   1  
2       NaN  19.0 NaN  100000.0  2012  january    xls   1

In [5]:
df.dtypes

structure_type       object
parcel               object
neighborhood         object
owner                object
issued               object
work description     object
permit               object
contractor           object
address              object
work_type            object
ward                 object
id                  float64
cost                 object
year                  int64
month                object
source               object
mn                    int64
dtype: object

In [72]:
# Missing data %
df.isna().sum()/len(df)*100

structure_type      78.311897
parcel               0.066139
neighborhood        80.851932
owner                0.022516
issued              24.160592
work description     7.665137
permit               0.000000
contractor          17.566351
address              0.000000
work_type           82.180350
ward                 1.218654
id                  27.359207
cost                 0.188568
year                 0.000000
month                0.000000
source               1.478990
mn                   0.000000
dtype: float64

In [73]:
no_cost = df[df.cost.isna()]

In [74]:
no_cost.to_csv('cost_na.csv',index=False)

In [75]:
df = df[~df.cost.isna()]

In [76]:
df[~df.address.str.contains('pittsburgh')]

structure_type                parcel  \
653                            NaN      0049c00068    00   
654                            NaN      0003l00296    00   
655                            NaN      0013c00307    00   
656                            NaN      0028b00131    00   
657                            NaN      0085f00077    00   
...                            ...                   ...   
68551  commercial - all other uses  0025-k-00164-0000-00   
68693                   commercial  0001-h-00388-0500-00   
69151  commercial - all other uses  0056-j-00045-0000-00   
70742  commercial - all other uses  0001-h-00388-0500-00   
70964  commercial - all other uses  0052-d-00141-0018-00   

                    neighborhood                                  owner  \
653                          NaN                      mark jessie davis   
654                          NaN              south high development lp   
655                          NaN              jlb development group llc   
656                          NaN                       schwartz linda b   
657                          NaN  kaufer david s & barbara j levine (w)   
...                          ...                                    ...   
68551             strip district                solara ventures vii llc   
68693  central business district                            renewal inc   
69151                  hazelwood                          progress fund   
70742  central business district                            renewal inc   
70964                  shadyside                             wang yixin   

                    issued                                   work description  \
653    2012-05-23 00:00:00  building permit only- abate condemnation of ex...   
654    2012-05-31 00:00:00  use of existing 4 story structure with 71 apar...   
655    2012-05-24 00:00:00  3-story single family attached dwelling with 2...   
656    2012-05-23 00:00:00  1,350 sq ft on 1st floor as retail sales and s...   
657    2012-05-22 00:00:00     building permit only- replacement of interior    
...                    ...                                                ...   
68551  2020-10-19t00:00:00  phased construction of 6 story structure with ...   
68693  2020-10-23t00:00:00  exterior equipment, interior equipment, ductwo...   
69151  2020-11-10t00:00:00  interior and exterior alterations to existing ...   
70742  2021-01-15t00:00:00  install new service, distribution equipment, p...   
70964  2021-01-26t00:00:00  replacement of existing front entryways/decks ...   

              permit                         contractor  \
653         09-01326              davis jesse carpentry   
654         09-06263         new-belle construction inc   
655         10-06326                 trends capital llc   
656         10-06383                sme real estate llc   
657         10-08612                   j francis co llc   
...              ...                                ...   
68551  bp-2020-04511                franjo construction   
68693  mp-2019-02480               r & b mechanical inc   
69151  bp-2020-05256         sota construction services   
70742  ep-2020-02913                 arrow electric inc   
70964  bp-2020-08288  liberty evergreen development llc   

                            address              work_type  ward     id  \
653                     273 45th st                    NaN   9.0    1.0   
654                 930 e carson st                    NaN  17.0    2.0   
655                    2355 holt st                    NaN  16.0    3.0   
656                 3533 forbes ave                    NaN   4.0    4.0   
657            1026 murray hill ave                    NaN  14.0    5.0   
...                             ...                    ...   ...    ...   
68551  no primary address specified       new construction     2  527.0   
68693  no primary address specified  addition / alteration     1  672.0   
69151  no primary address specified  addi

In [77]:
no_address = df[df.address.str.contains('no primary')]

In [78]:
no_address.to_csv('address_na.csv',index=False)

In [79]:
df = df[~df.address.str.contains('no primary')]

In [80]:
# Add pittsburgh to addresses with only streets
city = ', pittsburgh, pa'
f = lambda x: x+city if ',' not in x else x
df['address'] = df['address'].apply(lambda x:f(x))

In [81]:
df[~df.address.str.contains('pittsburgh')&~df.address.str.contains(' pa')]

structure_type            parcel neighborhood                 owner  \
3568             NaN  0003k00161000000          NaN  windom hill place lp   
53473            NaN  0023r00157000000          NaN        allegheny city   

                    issued                                   work description  \
3568   2013-06-03t00:00:00  10ft 3inches high retaining wall on easterly s...   
53473  2019-04-04t00:00:00  interior renovation of 960 sq ft on the ground...   

           permit                        contractor  \
3568   13-b-01323  sota construction services, inc.   
53473  19-b-00237   mascaro construction company lp   

                              address work_type  ward     id    cost  year  \
3568          22 windom hill pl, unit       NaN  17.0    7.0    4800  2013   
53473  6 allegheny square east, #101        NaN    22  151.0  149298  2019   

       month source  mn  
3568    june    API   6  
53473  april    API   4

In [82]:
df.loc[53473, 'address'] = '6 allegheny square east, #101, pittsburgh, pa'

In [83]:
df.loc[3568, 'address'] = '22 windom hill pl, pittsburgh, pa'

In [84]:
misspelled = df[df.address.str.contains('pittsbugrh')].index.values

In [85]:
for i in misspelled:
    df.loc[i,'address'] = df.loc[i,'address'].replace('pittsbugrh','pittsburgh')

In [86]:
df.duplicated().sum()

0

## Stopping point

In [121]:
df.to_csv('working.csv', index=False)

In [2]:
df = pd.read_csv('working.csv')

C:\Users\danie\anaconda_3\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (0,2,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [87]:
df[df.iloc[:,10].str.contains(r'[A-Za-z]')==True]

structure_type  parcel neighborhood  owner issued  work description  \
9508            NaN  parcel          NaN  owner    NaN  work description   

             permit  contractor                  address work_type  ward  \
9508  permit number  contractor  address, pittsburgh, pa       NaN  ward   

         id  cost  year   month source  mn  
9508  168.0  cost  2015  august    API   5

In [88]:
df.drop(index=9508, inplace = True)

In [89]:
df.ward.unique()

array(['6.0', '19.0', '15.0', '4.0', '1.0', '2.0', '29.0', '18.0', '31.0',
       '24.0', '14.0', '12.0', '20.0', '26.0', '10.0', '17.0', '13.0',
       '3.0', '22.0', '5.0', '21.0', '16.0', '7.0', '32.0', '11.0', '9.0',
       '27.0', '30.0', '23.0', '8.0', '25.0', '28.0', nan, '7', '14', '2',
       '5', '30', '1', '27', '13', '22', '12', '6', '11', '9', '19', '16',
       '4', '8', '15', '17', '10', '23', '20', '32', '31', '21', '25',
       '28', '24', '26', '29', '18', '3', '5/5', '2820', '19;20', '19;17',
       '26;25', '15;17', '10;6', '11;12', '21;22', '6;2', '9;14',
       '10;11;12', '21;27', '5;4', '24;23', '3;2', '18;29;32', '29;32',
       '19;32', '10;9', '18;30', '14;15', '20;28', '30;29', '6;9', '9;6',
       '2;10', '26;27'], dtype=object)

In [90]:
# Just keep the first one noted
def fix_ward(x):
    if pd.isna(x):
        return np.nan    
    elif ';' in x:
        return x.split(';')[0]
    elif '/' in x:
        return x.split('/')[0]
    elif '.' in x:
        return x.split('.')[0]
    elif '2820' in x:
        return '28'
    else:
        return x     

In [91]:
df['clean_ward'] = df['ward'].apply(lambda x:fix_ward(x))

In [92]:
df['cost'] = df['cost'].astype('float')

In [93]:
df.describe()

id          cost          year            mn
count  51445.000000  7.088700e+04  70887.000000  70887.000000
mean     434.795238  1.195332e+05   2017.061408      6.452763
std      303.099722  1.539839e+06      2.125455      3.348377
min        1.000000  0.000000e+00   2012.000000      1.000000
25%      170.000000  1.500000e+03   2016.000000      4.000000
50%      387.000000  6.533000e+03   2017.000000      6.000000
75%      672.000000  2.500000e+04   2019.000000     10.000000
max     1328.000000  1.900000e+08   2021.000000     12.000000

## Stopping point

In [19]:
df.to_csv('working.csv', index=False)

In [94]:
# Begin address cleaning, look for symbols
df[df['address'].str.findall(r'[^#a-zA-Z0-9, /]').str[0].notna()]

structure_type                parcel  \
5473                           NaN      0001c01662012500   
5573                           NaN      0001c01662f01900   
6843                           NaN      0001c01662h01200   
6918                           NaN      0001c01662009800   
8016                           NaN      0001c01662111300   
8681                           NaN      0001c01662021300   
8772                           NaN      0001c01662020900   
9173                           NaN      0001c01662009300   
12428                          NaN      0001c01662011200   
12429                          NaN      0001c01662024300   
13849                          NaN      0001c01662024200   
14849                          NaN      0001c01662014500   
15865                          NaN      0016r00156000000   
16062                          NaN      0001c01662171500   
17742                          NaN      0001c01662171500   
20847                          NaN      0016m00100000000   
21824                          NaN      0001h00256000000   
21886                          NaN      0016m00100000000   
21914                          NaN      0056k00248000000   
22306                          NaN      0126d00236000000   
23099                          NaN                134757   
23770                          NaN      0023l00080000000   
27602                          NaN      0001c01662023400   
44813                          NaN      0027h00074p00300   
45661                          NaN      0027h00074p00300   
50086                          NaN      0027h00074p00200   
53431                          NaN      0001c01662211300   
59967  commercial - all other uses  0001-c-01662-2211-00   
60903  commercial - all other uses  0001-c-01662-2211-00   
61036  commercial - all other uses  0001-c-01662-0410-00   
66775  commercial - all other uses  0025-f-00177-00g3-00   

                    neighborhood  \
5473                         NaN   
5573                         NaN   
6843                         NaN   
6918                         NaN   
8016                         NaN   
8681                         NaN   
8772                         NaN   
9173                         NaN   
12428                        NaN   
12429                        NaN   
13849                        NaN   
14849                        NaN   
15865                        NaN   
16062                        NaN   
17742                        NaN   
20847                        NaN   
21824                        NaN   
21886                        NaN   
21914                        NaN   
22306                        NaN   
23099                        NaN   
23770                        NaN   
27602                        NaN   
44813                        NaN   
45661                        NaN   
50086                        NaN   
53431                        NaN   
59967  central business district   
60903  central business district   
61036  central business district   
66775             strip district   

                                                   owner               issued  \
5473                                          ryan keefe  2014-01-24t00:00:00   
5573                                   betancourt carlos  2014-02-28t00:00:00   
6843                                anne and mike osborn  2014-06-06t00:00:00   
6918                                   barone alvin j jr  2014-06-13t00:00:00   
8016                     coogan david & leigh argentieri  2014-10-21t00:00:00   
8681                      lilan laishley, william harmon  2015-01-08t00:00:00   
8772                      miller harold d & sharon e (w)  2015-01-08t00:00:00   
9173                                     keefe timothy j  2015-03-13t00:00:00   
12428                                  burstein robert h                  NaN   
12429                                  burstein robert h                  NaN   
13849                                  burstein robert 

In [95]:
# Prefer just single # to refer to anything regarding "unit" as this is common and known by Geocoders
df['address'] = df['address'].str.replace('#unit','#')

In [96]:
df['address'] = df['address'].str.replace('##','#')

In [97]:
# Find digit-letter units and kill the dash, i.e., make 12-e into 12e
pat = r'([0-9][-][a-zA-Z])'
df[df['address'].str.contains(pat)]

C:\Users\danie\anaconda_3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


structure_type                parcel  \
5473                           NaN      0001c01662012500   
5573                           NaN      0001c01662f01900   
6843                           NaN      0001c01662h01200   
6918                           NaN      0001c01662009800   
8016                           NaN      0001c01662111300   
8681                           NaN      0001c01662021300   
8772                           NaN      0001c01662020900   
9173                           NaN      0001c01662009300   
12428                          NaN      0001c01662011200   
12429                          NaN      0001c01662024300   
13849                          NaN      0001c01662024200   
14849                          NaN      0001c01662014500   
16062                          NaN      0001c01662171500   
17742                          NaN      0001c01662171500   
27602                          NaN      0001c01662023400   
44813                          NaN      0027h00074p00300   
50086                          NaN      0027h00074p00200   
53431                          NaN      0001c01662211300   
59967  commercial - all other uses  0001-c-01662-2211-00   
60903  commercial - all other uses  0001-c-01662-2211-00   
61036  commercial - all other uses  0001-c-01662-0410-00   

                    neighborhood                             owner  \
5473                         NaN                        ryan keefe   
5573                         NaN                 betancourt carlos   
6843                         NaN              anne and mike osborn   
6918                         NaN                 barone alvin j jr   
8016                         NaN   coogan david & leigh argentieri   
8681                         NaN    lilan laishley, william harmon   
8772                         NaN    miller harold d & sharon e (w)   
9173                         NaN                   keefe timothy j   
12428                        NaN                 burstein robert h   
12429                        NaN                 burstein robert h   
13849                        NaN                 burstein robert h   
14849                        NaN  caruso thomas b caruso saverio t   
16062                        NaN                   sapotichne sean   
17742                        NaN       baker sidney & judith r (w)   
27602                        NaN   bullano joseph v & leslie a (w)   
44813                        NaN       barone loretta & robert (h)   
50086                        NaN             norman  & gail childs   
53431                        NaN            adler niki kapsambelis   
59967  central business district     hoglund j david & loretta (w)   
60903  central business district     hoglund j david & loretta (w)   
61036  central business district                       zeng xiaosi   

                    issued                                   work description  \
5473   2014-01-24t00:00:00                     interior condo renovations-12e   
5573   2014-02-28t00:00:00                                  condo renovations   
6843   2014-06-06t00:00:00  interior renovations to condo unit 12 h on 12t...   
6918   2014-06-13t00:00:00  interior renovations to kitchen in condo unit ...   
8016   2014-10-21t00:00:00  interior renovations to condo unit 11m in 27 s...   
8681   2015-01-08t00:00:00  interior renovations to kitchen and bath in co...   
8772   2015-01-08t00:00:00  interior renovations to bath in condo 20j in 2...   
9173   2015-03-13t00:00:00  interior renovations to kitchen and bath in un...   
12428                  NaN  interior demolition to unit 24c in 25 story st...   
12429                  NaN  interior demolition to unit 24c in 25 story st...   
13849            11/9/2015                                           unit 24b   
14849                  NaN  approximately 325 sq ft of interior renovation...   
16062            2/16/2016  interior renovations to kitchen and bath in co...   
17742                  N

In [98]:
df[df['address'].str.contains(pat)].index.values

C:\Users\danie\anaconda_3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


array([ 5473,  5573,  6843,  6918,  8016,  8681,  8772,  9173, 12428,
       12429, 13849, 14849, 16062, 17742, 27602, 44813, 50086, 53431,
       59967, 60903, 61036], dtype=int64)

In [59]:
df.loc[5473,'address'].replace('-','')

'320 fort duquesne blvd 12e, pittsburgh, pa'

In [99]:
for i in df[df['address'].str.contains(pat)].index.values:
    df.loc[i,'address'] = df.loc[i,'address'].replace('-','')

In [100]:
df[df['address'].str.findall(r'[^#a-zA-Z0-9, /]').str[0].notna()]

structure_type                parcel    neighborhood  \
15865                          NaN      0016r00156000000             NaN   
20847                          NaN      0016m00100000000             NaN   
21824                          NaN      0001h00256000000             NaN   
21886                          NaN      0016m00100000000             NaN   
21914                          NaN      0056k00248000000             NaN   
22306                          NaN      0126d00236000000             NaN   
23099                          NaN                134757             NaN   
23770                          NaN      0023l00080000000             NaN   
45661                          NaN      0027h00074p00300             NaN   
66775  commercial - all other uses  0025-f-00177-00g3-00  strip district   

                                                   owner               issued  \
15865  board of public education of the schl dist of ...             2/8/2016   
20847                      school district of pittsburgh             6/7/2016   
21824                      pnc bank national association             7/6/2016   
21886                      school district of pittsburgh             7/7/2016   
21914                            hazelwood second ave lp             7/8/2016   
22306                         darnall lamoyne william jr            7/18/2016   
23099                       penn liberty holding company            9/13/2016   
23770                                 coa north side llc                  NaN   
45661                          robert and loretta barone  2018-06-27t00:00:00   
66775                        berger lee j & meghan f (w)  2020-07-21t00:00:00   

                                        work description          permit  \
15865  to replace permit #14-f-00188. see #14-f-00188...      16-f-00034   
20847                              fire alarm alteration      16-f-00181   
21824  work related to 16-b-00517 relocation of offic...      16-f-00209   
21886                                    install outlets      16-e-01772   
21914                                          reconnect      16-e-01805   
22306                                       total rewire      16-e-01926   
23099                               generators for event      16-e-02566   
23770                     install hood supression system      16-s-00267   
45661         fixtures and outlets related to 18-b-01938      18-e-01733   
66775        add new sprinkler heads to existing system.  ssp-2020-10123   

                         contractor  \
15865                           NaN   
20847                right electric   
21824           lighthouse electric   
21886             wheels mechanical   
21914                           NaN   
22306                           NaN   
23099                           NaN   
23770       fire control serv. inc.   
45661        schreckengost electric   
66775  metropolitan fire protection   

                                                 address work_type  ward  \
15865         810 rockland ave, pittsburgh< pa, pa 15216       NaN  19.0   
20847           590 -595 crane ave, pittsburgh, pa 15216       NaN  19.0   
21824             300 -314 5th ave, pittsburgh, pa 15222       NaN   2.0   
21886           590 -595 crane ave, pittsburgh, pa 15216       NaN  19.0   
21914           5011 -5013 2nd ave, pittsburgh, pa 15207       NaN  15.0   
22306      205 -207 n homewood ave, pittsburgh, pa 15208       NaN  14.0   
23099                         penn & 7th, pittsburgh, pa       NaN   2.0   
23770           40 -44 w north ave, pittsburgh, pa 15212       NaN    22   
45661  146 n bellefield ave, #ph-3, pittsburgh, pa 15213       NaN     7   
66775         2901 g-3 smallman st, pittsburgh, pa 15201       NaN     6   

          id     cost  year      month source  mn clean_ward  
15865    NaN      0.0  2016   february   xslx   2         19  
20847    NaN   9000.0  2016       june   xslx   6         19 

In [106]:
# Narrow change for weird address
df.loc[15865,'address'] = df.loc[15865,'address'].replace('< pa','')

In [107]:
# Narrow change for weird address
df.loc[45661,'address'] = df.loc[45661,'address'].replace('#ph-3','# ph3')

In [101]:
# Narrow change for weird address
df.loc[66775,'address'] = df.loc[66775,'address'].replace('g-3','g3')

In [102]:
multiaddress = df[df['address'].str.findall(r'[^#a-zA-Z0-9, /]').str[0].notna()].index.values

In [103]:
def take_first(x):
    if " -" in x:
        y = ' '+x.split()[1]
        first = x.replace(y, '')
        return first
    else:
        return x

In [104]:
# Take addresses like "100 -105 n avenue" and just make them "100 n avenue" for simplicity
for addr in multiaddress:
    df.loc[addr, 'address'] = take_first(df.loc[addr, 'address'])

In [108]:
df[df['address'].str.findall(r'[^#a-zA-Z0-9, /]').str[0].notna()]

structure_type  parcel neighborhood                         owner  \
23099            NaN  134757          NaN  penn liberty holding company   

          issued      work description      permit contractor  \
23099  9/13/2016  generators for event  16-e-02566        NaN   

                          address work_type ward  id   cost  year      month  \
23099  penn & 7th, pittsburgh, pa       NaN  2.0 NaN  450.0  2016  september   

      source  mn clean_ward  
23099   xslx   9          2

In [109]:
# Look for two+ white spaces to address gaps
df[df.loc[:,'address'].str.contains(r'([ ]){2,}')]

C:\Users\danie\anaconda_3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


structure_type            parcel neighborhood  \
52458            NaN  0123n00322000000          NaN   
56113            NaN  0002m00065000000          NaN   

                                            owner               issued  \
52458  harding john daniel jr & emilykaufmann (w)  2019-03-06 00:00:00   
56113                mercy hospital of pittsburgh  2019-05-21t00:00:00   

                                        work description      permit  \
52458  construction of soil nail retaining wall and i...  18-l-00015   
56113       install fixtures on existing branch circuits  19-e-00656   

                               contractor  \
52458  geostabilization international llc   
56113             hanlon electric company   

                                                 address work_type  ward  \
52458     6512     jackson st,    , pittsburgh, pa 15206       NaN  11.0   
56113  1401     blvd of the allies  ,    , pittsburgh...       NaN   1.0   

          id      cost  year  month source  mn clean_ward  
52458    NaN  300000.0  2019  march    NaN   3         11  
56113  614.0   25900.0  2019    may    API   5          1

In [110]:
# Only 2! just change in line
df.loc[52458,'address'] = '6512 jackson st, pittsburgh, pa 15206'

In [111]:
df.loc[56113,'address'] = '1401 blvd of the allies, pittsburgh, pa 15219'

In [112]:
df[df.loc[:,'address'].str.contains(r" b(ui)?ld(in)?g ")]

C:\Users\danie\anaconda_3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


structure_type            parcel neighborhood  \
47596            NaN  0120d00024000000          NaN   

                                owner               issued  \
47596  marlene  a curley & urbash llc  2018-09-26t00:00:00   

                                work description        permit contractor  \
47596  192 sq. ft. wall mounted business id sign  18-occ-00744        NaN   

                                            address work_type  ward     id  \
47596  6411 butler st, bldg b, pittsburgh, pa 15206       NaN  10.0  902.0   

       cost  year      month source  mn clean_ward  
47596   0.0  2018  september    API   9         10

In [113]:
# clear hanging white space
df.loc[:,'address'] = df.loc[:,'address'].str.replace(r' $', '')

In [114]:
df.loc[:,'address'].str.contains(r' $').sum()

0

In [23]:
# likely abbreviations flanked by commas or whitespace
full = {' rd,':' road,', ' ave,':' avenue,',' st,':' street,', ' blvd,':' boulevard,',
 ' ct,':' court,', ' pl,':' place,', ' dr,':' drive,', ' ln,':' lane,', ' ctr,': ' center,',
       ' rd ':' road,', ' ave ':' avenue ',' st ':' street ', ' blvd ':' boulevard,',
 ' ct ':' court,', ' pl ':' place,', ' dr ':' drive,', ' ln ':' lane,', ' ctr ': ' center,'}

In [24]:
def complete_street(x):
    for i in full.keys():
        if i in x:
            x = x.replace(i, full[i])
    return x

In [117]:
# likely abbreviations flanked by commas or whitespace
direc = {' n ':' north ', ' e ':' east ',' s ':' south ', ' w ':' west '}

In [123]:
def complete_direc(x):
    for i in direc.keys():
        if i in x:
            x = x.replace(i, direc[i])
    return x

In [25]:
# Apply the func to cleanse abbreviations
df.loc[:,'address'] = df.loc[:,'address'].apply(lambda x: complete_street(x))

C:\Users\danie\anaconda_3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [135]:
nth = {' 1st':' first', ' 2nd':' second',' 3rd':' third', ' 4th':' fourth', ' 5th':' fifth',
      ' 6th':' sixth', ' 7th':' seventh', ' 8th':' eighth', ' 9th':' ninth'}

In [136]:
def complete_nth(x):
    for i in nth.keys():
        if i in x:
            x = x.replace(i, nth[i])
    return x

In [137]:
df.loc[:,'address'] = df.loc[:,'address'].apply(lambda x: complete_nth(x))

In [138]:
df.loc[:,'address']

0                     3418 penn avenue, pittsburgh, pa
1                435 sweetbriar street, pittsburgh, pa
2            437 sweetbriar street 435, pittsburgh, pa
3           4656 browns hill road,4634, pittsburgh, pa
4                    4400 fifth avenue, pittsburgh, pa
                             ...                      
71057          323 denniston street, pittsburgh, 15206
71058            211 26th street, pittsburgh, pa 15222
71059              5900 penn avenue, pittsburgh, 15206
71060    211 north whitfield street, pittsburgh, 15206
71061          323 denniston street, pittsburgh, 15206
Name: address, Length: 70887, dtype: object

## Stopping point

In [11]:
df.to_csv('working.csv', index=False)

In [2]:
df = pd.read_csv('working.csv')

C:\Users\danie\anaconda_3\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (0,2,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.iloc[:,[0,2,9]]

structure_type    neighborhood              work_type
0                NaN             NaN                    NaN
1                NaN             NaN                    NaN
2                NaN             NaN                    NaN
3                NaN             NaN                    NaN
4                NaN             NaN                    NaN
...              ...             ...                    ...
70882    residential       shadyside  addition / alteration
70883     commercial  strip district  addition / alteration
70884     commercial    east liberty  addition / alteration
70885     commercial    east liberty       minor alteration
70886    residential       shadyside  addition / alteration

[70887 rows x 3 columns]

In [9]:
df.iloc[:,0].replace(np.nan,'not_listed')

0         not_listed
1         not_listed
2         not_listed
3         not_listed
4         not_listed
            ...     
70882    residential
70883     commercial
70884     commercial
70885     commercial
70886    residential
Name: structure_type, Length: 70887, dtype: object

In [10]:
for i in [0,2,9]:
    df.iloc[:,i]= df.iloc[:,i].replace(np.nan,'not_listed')

In [28]:
pat = r'^[a-zA-Z 0,]'

In [29]:
df[df.loc[:,'address'].str.contains(pat)].index.values

array([20600, 28178], dtype=int64)

In [37]:
df[df.loc[:,'address'].str.contains(pat)];

In [16]:
df[df.loc[:,'address'].str.contains(pat)].to_csv('addr_number_na.csv',index=False)

In [39]:
df = df[~df.loc[:,'address'].str.contains(pat)]

In [41]:
len(df)

70812

In [139]:
df.to_csv('working.csv', index=False)

In [12]:
# Recheck for mixed types after clearing NaNs
df = pd.read_csv('working.csv')

## Split addresses to see about consolidating where possible

In [66]:
df = pd.read_csv('working.csv')

In [47]:
df.loc[:,'address'] = df.loc[:,'address'].str.replace('boulevard,of the allies,', 'boulevard of the allies,')

In [48]:
df.head(2)

structure_type            parcel neighborhood                        owner  \
0     not_listed  0048s00284000000   not_listed  pfohl donald e & lucy g (w)   
1     not_listed  0006r00206000000   not_listed            p7 investments lp   

                issued                                   work description  \
0  2012-01-30 00:00:00  interior fitout for yoga studio\n-pittsburgh b...   
1  2012-01-04 00:00:00  new 3 story single family dwelling with 2 car ...   

       permit           contractor                                address  \
0  11-b-03000  nh construction llc       3418 penn avenue, pittsburgh, pa   
1  11-b-03019  heartland homes inc  435 sweetbriar street, pittsburgh, pa   

    work_type  ward  id      cost  year    month source  mn  clean_ward  
0  not_listed   6.0 NaN  200000.0  2012  january    xls   1         6.0  
1  not_listed  19.0 NaN  100000.0  2012  january    xls   1        19.0

In [134]:
df['split_addr'] = df.loc[:,'address'].str.split(',')

In [135]:
x = 0
for i in df['address'].str.split(','):
    if len(i) > x:
        x = len(i)

In [136]:
x

4

In [137]:
df['addr1'] = df.loc[:,'split_addr'].apply(lambda x: x[0] if len(x)>0 else '')

In [138]:
df['addr2'] = df.loc[:,'split_addr'].apply(lambda x: x[1] if len(x)>1 else '')
df['addr3'] = df.loc[:,'split_addr'].apply(lambda x: x[2] if len(x)>2 else '')
df['addr4'] = df.loc[:,'split_addr'].apply(lambda x: x[3] if len(x)>3 else '')

In [9]:
sorted(df.addr2.unique());

In [127]:
# isolate addr2 components that need fixing
df[df.loc[:,'addr2'].str.contains(r'^[a-z]')]

Empty DataFrame
Columns: [structure_type, parcel, neighborhood, owner, issued, work description, permit, contractor, address, work_type, ward, id, cost, year, month, source, mn, clean_ward, split_addr, addr1, addr2, addr3, addr4]
Index: []

[0 rows x 23 columns]

In [56]:
# Get rid of these 'a' and 'cc' type additions, they mean nothing to addresses
df.loc[:,'address'] = df.loc[:,'address'].str.replace(r',[a-z]{1},', ',')

In [67]:
df.loc[:,'address'] = df.loc[:,'address'].str.replace(r',[a-z]{2},', ',')

In [75]:
# Narrow fixes for things I probably messed up (drive, blvd...)
fixes = ['boulevard,of the allies avenue,',
'center,center,',
'boulevard,of the allies boulevard,',
'drive,drive,']

In [83]:
df.loc[:,'address'] = df.loc[:,'address'].str.replace(fixes[0], 'boulevard of the allies,')
df.loc[:,'address'] = df.loc[:,'address'].str.replace(fixes[1], 'center,')
df.loc[:,'address'] = df.loc[:,'address'].str.replace(fixes[2], 'boulevard of the allies,')
df.loc[:,'address'] = df.loc[:,'address'].str.replace(fixes[3], 'drive,')

## Stopping point

In [105]:
df.to_csv('working.csv', index=False)

In [35]:
df = pd.read_csv('working.csv')

In [36]:
sorted(df.addr2.unique())

[' ',
 ' # ph3',
 ' #1',
 ' #1004',
 ' #1006',
 ' #101',
 ' #1015',
 ' #102',
 ' #103',
 ' #104',
 ' #105',
 ' #1250',
 ' #13',
 ' #1404',
 ' #1415',
 ' #142',
 ' #14e',
 ' #15',
 ' #15212',
 ' #15c',
 ' #165',
 ' #17',
 ' #175',
 ' #17o',
 ' #18',
 ' #1801',
 ' #1902',
 ' #19a',
 ' #2',
 ' #20',
 ' #2001',
 ' #201',
 ' #202',
 ' #203',
 ' #204',
 ' #206',
 ' #21',
 ' #210',
 ' #21f',
 ' #21m',
 ' #22',
 ' #23d',
 ' #24c',
 ' #24h',
 ' #25',
 ' #28',
 ' #2b',
 ' #2p',
 ' #3',
 ' #301',
 ' #302',
 ' #303',
 ' #304',
 ' #306',
 ' #308',
 ' #310',
 ' #311',
 ' #33',
 ' #3a',
 ' #3b',
 ' #3p',
 ' #4',
 ' #401',
 ' #402',
 ' #403',
 ' #406',
 ' #408',
 ' #411',
 ' #412',
 ' #414',
 ' #450',
 ' #4a',
 ' #4jk',
 ' #5',
 ' #503',
 ' #505',
 ' #509',
 ' #512',
 ' #519',
 ' #55',
 ' #57',
 ' #6',
 ' #601',
 ' #602',
 ' #604',
 ' #606',
 ' #615',
 ' #616',
 ' #6g',
 ' #6k',
 ' #7',
 ' #701',
 ' #705',
 ' #707',
 ' #711',
 ' #716',
 ' #717',
 ' #7e',
 ' #8',
 ' #806',
 ' #81',
 ' #840c',
 ' #850c'

In [104]:
# has 'pittsburgh,' but not PA 
noPA = df[~df.loc[:,'address'].str.contains(' pa') & df.loc[:,'address'].str.contains('pittsburgh,')].index.values

In [105]:
noPA

array([54300, 54301, 54302, ..., 70809, 70810, 70811], dtype=int64)

In [110]:
df.loc[noPA[0],'address'].replace('pittsburgh,', 'pittsburgh, pa')

'1931 sidney street, pittsburgh, pa 15203'

In [109]:
# Place PA in address
for i in noPA:
    df.loc[i,'address'] = df.loc[i,'address'].replace(' pittsburgh,', ' pittsburgh, pa')

In [111]:
noComma = df[~df.loc[:,'address'].str.contains(' pa') & ~df.loc[:,'address'].str.contains('pittsburgh,')].index.values

In [112]:
# Place PA in address
for i in noComma:
    df.loc[i,'address'] = df.loc[i,'address'].replace(' pittsburgh', ' pittsburgh, pa')

In [113]:
df[~df.addr2.str.contains('#') & ~df.addr2.str.contains('pittsburgh')][:50]

structure_type            parcel neighborhood  \
3        not_listed  0088r00175000000   not_listed   
5461     not_listed  0001c01662012500   not_listed   
5561     not_listed  0001c01662f01900   not_listed   
6459     not_listed  0010f00210000000   not_listed   
6460     not_listed  0010f00210000000   not_listed   
6468     not_listed  0010f00210000000   not_listed   
6562     not_listed  0010f00210000000   not_listed   
6563     not_listed  0010f00210000000   not_listed   
6574     not_listed  0010f00210000000   not_listed   
6575     not_listed  0010f00210000000   not_listed   
6580     not_listed  0010f00210000000   not_listed   
6581     not_listed  0010f00210000000   not_listed   
6679     not_listed  0010f00210000000   not_listed   
6696     not_listed  0010f00210000000   not_listed   
6697     not_listed  0010f00210000000   not_listed   
6714     not_listed  0010f00210000000   not_listed   
6831     not_listed  0001c01662h01200   not_listed   
6906     not_listed  0001c01662009800   not_listed   
7122     not_listed  0088r00175000000   not_listed   
7478     not_listed  0001g00224160400   not_listed   
7870     not_listed  0001g00224090200   not_listed   
8002     not_listed  0001c01662111300   not_listed   
8463     not_listed  0001g00224140400   not_listed   
8667     not_listed  0001c01662021300   not_listed   
8758     not_listed  0001c01662020900   not_listed   
8872     not_listed  0088r00175000000   not_listed   
8896     not_listed  0001g00224150600   not_listed   
9159     not_listed  0001c01662009300   not_listed   
9484     not_listed  0116l00076000000   not_listed   
9485     not_listed  0023g00135000000   not_listed   
9488     not_listed  0028h00172000000   not_listed   
9489     not_listed  0022p00271000000   not_listed   
9490     not_listed  0126j00303000000   not_listed   
9491     not_listed  0050f00167000000   not_listed   
9492     not_listed  0049f00149000000   not_listed   
9493     not_listed  0024g00197000000   not_listed   
9494     not_listed  0001d00169000000   not_listed   
9495     not_listed  0012m00163000000   not_listed   
9496     not_listed  0023f00229000000   not_listed   
9497     not_listed  0009k00057000000   not_listed   
9499     not_listed  0035a00280000000   not_listed   
9500     not_listed  0002e00184000000   not_listed   
9501     not_listed  0002e00184000a00   not_listed   
9504     not_listed  0049n00088000a00   not_listed   
9505     not_listed  0046l00225000000   not_listed   
9506     not_listed  0005d00223000000   not_listed   
9507     not_listed  0126a00109000000   not_listed   
9508     not_listed  0008f00118000000   not_listed   
9509     not_listed  0126a00108000000   not_listed   
9510     not_listed  0120n00120000000   not_listed   

                                                  owner               issued  \
3                                browns hill venture lp  2012-01-17 00:00:00   
5461                                         ryan keefe  2014-01-24t00:00:00   
5561                                  betancourt carlos  2014-02-28t00:00:00   
6459               housing authority city of pittsburgh  2014-05-05t00:00:00   
6460               housing authority city of pittsburgh  2014-05-05t00:00:00   
6468               housing authority city of pittsburgh  2014-05-05t00:00:00   
6562               housing authority city of pittsburgh  2014-05-05t00:00:00   
6563               housing authority city of pittsburgh  2014-05-05t00:00:00   
6574               housing authority city of pittsburgh  2014-05-05t00:00:00   
6575               housing authority city of pittsburgh  2014-05-05t00:00:00   
6580               housing authority city of pittsburgh  2014-05-05t00:00:00   
6581               housing authority city of pittsburgh  2014-05-05t00:00:00   
6679               housing authority city of pittsburgh  2014-05-05t00:00:00   
6696               housing authority city of pittsburgh  2014-05-05t00:00:00   
6697               housing authority city of pitts

In [116]:
df.loc[9484, 'address']

'3608 colby, pittsburgh, pa 15214'

In [115]:
# Purge double commas
df.address = df.address.str.replace(',,', ',')

## Stopping point

In [41]:
df.to_csv('working.csv', index=False)

In [2]:
df = pd.read_csv('working.csv')

In [78]:
df.loc[:,'addr2'].unique()

array([' pittsburgh', '4634', '12e', '19f', '179', '187', '171', '9',
       '17', '105', '142', '117', '129', '165', '1', '25', '153', '12h',
       '9h', '1604', '902', '11m', '1404', '21c', '20i', '1506', '9c',
       ' #6', ' homestead', ' #711', ' #gg', ' #jj', ' #19a', ' #ii',
       ' #hh', ' #411', ' #1006', ' #7', ' #24c', ' #4a', ' #707',
       ' carnegie', ' #u', ' #o', ' #p', ' #q', ' #r', ' #s', ' #t',
       ' #v', ' #w', ' #x', ' #y', ' #z', ' #aa', ' #24h', ' #14e',
       ' #1404', ' #165', ' #142', ' #33', ' #81', ' #57', ' #n', ' #d',
       ' #a', ' #e', ' #i', ' #g', ' #b', ' #c', ' #h', ' #l', ' #m',
       ' #j', ' #k', ' #17o', ' #906', ' #601', ' #519', ' #503', ' #505',
       ' #616', ' #15212', ' #101', ' #102', ' #103', ' #104', ' #201',
       ' #202', ' #203', ' #204', ' #301', ' #303', ' #304', ' #401',
       ' #18', ' #13', ' #414', ' #306', ' #rear', ' #512', ' #5', ' #2',
       ' #21', ' #55', ' #602', ' #406', ' #b103', ' rear ', ' #f',
       ' #

In [10]:
# replace dict
repl = {' unit ': ' # ', ' bldg ':' building ', ' rear rear':'', ' #res':'', ' #onechatham':'',
       ' ste ': ' suite ', ' #parking': '', ' #f8 ud': '#f8', '# ':'#'}

In [11]:
def complete_street(x, dct):
    for i in dct.keys():
        if i in x:
            x = x.replace(i, dct[i])
    return x

In [12]:
# Apply the func to cleanse abbreviations
df.loc[:,'address'] = df.loc[:,'address'].apply(lambda x: complete_street(x, repl))

In [14]:
df[df.address.str.contains("''")]

Empty DataFrame
Columns: [structure_type, parcel, neighborhood, owner, issued, work description, permit, contractor, address, work_type, ward, id, cost, year, month, source, mn, clean_ward, split_addr, addr1, addr2, addr3, addr4]
Index: []

[0 rows x 23 columns]

In [79]:
df.loc[:,'addr2'].unique()

array([' pittsburgh', '4634', '12e', '19f', '179', '187', '171', '9',
       '17', '105', '142', '117', '129', '165', '1', '25', '153', '12h',
       '9h', '1604', '902', '11m', '1404', '21c', '20i', '1506', '9c',
       ' #6', ' homestead', ' #711', ' #gg', ' #jj', ' #19a', ' #ii',
       ' #hh', ' #411', ' #1006', ' #7', ' #24c', ' #4a', ' #707',
       ' carnegie', ' #u', ' #o', ' #p', ' #q', ' #r', ' #s', ' #t',
       ' #v', ' #w', ' #x', ' #y', ' #z', ' #aa', ' #24h', ' #14e',
       ' #1404', ' #165', ' #142', ' #33', ' #81', ' #57', ' #n', ' #d',
       ' #a', ' #e', ' #i', ' #g', ' #b', ' #c', ' #h', ' #l', ' #m',
       ' #j', ' #k', ' #17o', ' #906', ' #601', ' #519', ' #503', ' #505',
       ' #616', ' #15212', ' #101', ' #102', ' #103', ' #104', ' #201',
       ' #202', ' #203', ' #204', ' #301', ' #303', ' #304', ' #401',
       ' #18', ' #13', ' #414', ' #306', ' #rear', ' #512', ' #5', ' #2',
       ' #21', ' #55', ' #602', ' #406', ' #b103', ' rear ', ' #f',
       ' #

In [80]:
df[df.addr2.str.contains(r'[0-9]{3,4}') & ~df.addr2.str.contains(r'#|suite')]

structure_type            parcel neighborhood  \
3        not_listed  0088r00175000000   not_listed   
6459     not_listed  0010f00210000000   not_listed   
6460     not_listed  0010f00210000000   not_listed   
6468     not_listed  0010f00210000000   not_listed   
6574     not_listed  0010f00210000000   not_listed   
6575     not_listed  0010f00210000000   not_listed   
6580     not_listed  0010f00210000000   not_listed   
6581     not_listed  0010f00210000000   not_listed   
6679     not_listed  0010f00210000000   not_listed   
6714     not_listed  0010f00210000000   not_listed   
7122     not_listed  0088r00175000000   not_listed   
7478     not_listed  0001g00224160400   not_listed   
7870     not_listed  0001g00224090200   not_listed   
8463     not_listed  0001g00224140400   not_listed   
8872     not_listed  0088r00175000000   not_listed   
8896     not_listed  0001g00224150600   not_listed   

                                     owner               issued  \
3                   browns hill venture lp  2012-01-17 00:00:00   
6459  housing authority city of pittsburgh  2014-05-05t00:00:00   
6460  housing authority city of pittsburgh  2014-05-05t00:00:00   
6468  housing authority city of pittsburgh  2014-05-05t00:00:00   
6574  housing authority city of pittsburgh  2014-05-05t00:00:00   
6575  housing authority city of pittsburgh  2014-05-05t00:00:00   
6580  housing authority city of pittsburgh  2014-05-05t00:00:00   
6581  housing authority city of pittsburgh  2014-05-05t00:00:00   
6679  housing authority city of pittsburgh  2014-05-05t00:00:00   
6714  housing authority city of pittsburgh  2014-05-05t00:00:00   
7122                browns hill venture lp  2014-07-09t00:00:00   
7478                       jonathan moritz  2014-09-26t00:00:00   
7870                  cornell ray & denise  2014-10-30t00:00:00   
8463          151 first side associates lp  2014-12-05t00:00:00   
8872                browns hill venture lp  2015-02-05t00:00:00   
8896          151 first side associates lp  2015-02-04t00:00:00   

                                       work description      permit  \
3     tenant fitout of 2008 sq ft retail space in on...  11-b-03148   
6459         windows & masonary repairs (building # 15)  14-b-01065   
6460         windows & masonary repairs (building # 16)  14-b-01066   
6468         windows & masonary repairs (building # 14)  14-b-01042   
6574                         windows & masonary repairs  14-b-01005   
6575                         windows & masonary repairs  14-b-01008   
6580                         windows & masonary repairs  14-b-01006   
6581                         windows & masonary repairs  14-b-01007   
6679         windows & masonary repairs (building # 13)  14-b-01040   
6714         windows & masonary repairs (building # 12)  14-b-01039   
7122                                      tenant fitout  14-b-01203   
7478  interior fitout of condo unit 1604 for a singl...  14-b-03114   
7870  interior renovations to single family condo un...  14-b-03575   
8463  fitout of condo unit 1404 as a single family d...  14-b-03902   
8872                 interior renovation, tenant fitout  14-b-04037   
8896  tenant fit out of unit #1506 as single family ...  15-b-00032   

                        contractor  \
3           gem construction, inc.   
6459            air technology inc   
6460            air technology inc   
6468            air technology inc   
6574            air technology inc   
6575            air technology inc   
6580            air technology inc   
6581            air technology inc   
6679            air technology inc   
6714            air technology inc   
7122              absolute win inc   
7478                           NaN   
7870              jcp construction   
8463                  build rx inc   
8872     metro classic builder inc   
8896  mistick construction company   

                                           address   work_type  ...  year  \
3       4656 b

In [81]:
idx = df[df.addr2.str.contains(r'[0-9]{3,4}') & ~df.addr2.str.contains(r'#|suite')].index.values

In [82]:
idx

array([   3, 6459, 6460, 6468, 6574, 6575, 6580, 6581, 6679, 6714, 7122,
       7478, 7870, 8463, 8872, 8896], dtype=int64)

In [83]:
# Place '#' unit signifier before second address number
for i in idx:
    df.loc[i,'addr2'] = "#"+df.loc[i,'addr2']

In [84]:
",".join(df.iloc[3,19:22])

'4656 browns hill road,#4634, pittsburgh'

In [85]:
df.loc[3,'address']

'4656 browns hill road,4634, pittsburgh, pa'

In [86]:
for i in idx:
    df.loc[i,'address'] = ",".join(df.iloc[i,19:22])

In [94]:
df[~df.addr2.str.contains(r'#|suite|pitts|home|carn|apt|rear|pa|build')]

structure_type            parcel neighborhood  \
5461     not_listed  0001c01662012500   not_listed   
5561     not_listed  0001c01662f01900   not_listed   
6562     not_listed  0010f00210000000   not_listed   
6563     not_listed  0010f00210000000   not_listed   
6696     not_listed  0010f00210000000   not_listed   
6697     not_listed  0010f00210000000   not_listed   
6831     not_listed  0001c01662h01200   not_listed   
6906     not_listed  0001c01662009800   not_listed   
8002     not_listed  0001c01662111300   not_listed   
8667     not_listed  0001c01662021300   not_listed   
8758     not_listed  0001c01662020900   not_listed   
9159     not_listed  0001c01662009300   not_listed   

                                     owner               issued  \
5461                            ryan keefe  2014-01-24t00:00:00   
5561                     betancourt carlos  2014-02-28t00:00:00   
6562  housing authority city of pittsburgh  2014-05-05t00:00:00   
6563  housing authority city of pittsburgh  2014-05-05t00:00:00   
6696  housing authority city of pittsburgh  2014-05-05t00:00:00   
6697  housing authority city of pittsburgh  2014-05-05t00:00:00   
6831                  anne and mike osborn  2014-06-06t00:00:00   
6906                     barone alvin j jr  2014-06-13t00:00:00   
8002       coogan david & leigh argentieri  2014-10-21t00:00:00   
8667        lilan laishley, william harmon  2015-01-08t00:00:00   
8758        miller harold d & sharon e (w)  2015-01-08t00:00:00   
9159                       keefe timothy j  2015-03-13t00:00:00   

                                       work description      permit  \
5461                     interior condo renovations-12e  14-b-00143   
5561                                  condo renovations  14-b-00106   
6562                         windows & masonary repairs  14-b-00981   
6563                         windows & masonary repairs  14-b-00982   
6696                         windows & masonary repairs  14-b-00980   
6697                         windows & masonary repairs  14-b-00983   
6831  interior renovations to condo unit 12 h on 12t...  14-b-01256   
6906  interior renovations to kitchen in condo unit ...  14-b-01776   
8002  interior renovations to condo unit 11m in 27 s...  14-b-03439   
8667  interior renovations to kitchen and bath in co...  15-b-00040   
8758  interior renovations to bath in condo 20j in 2...  15-b-00041   
9159  interior renovations to kitchen and bath in un...  15-b-00621   

                             contractor  \
5461  randy stebler general contracting   
5561   fjw investment inc cabinet world   
6562                 air technology inc   
6563                 air technology inc   
6696                 air technology inc   
6697                 air technology inc   
6831               delaney's decorating   
6906                         renovision   
8002                         renovision   
8667                         renovision   
8758                         renovision   
9159  randy stebler general contracting   

                                              address   work_type  ...  year  \
5461  320 fort duquesne boulevard,12e, pittsburgh, pa  not_listed  ...  2014   
5561  320 fort duquesne boulevard,19f, pittsburgh, pa  not_listed  ...  2014   
6562              2249 somers drive,9, pittsburgh, pa  not_listed  ...  2014   
6563             2267 somers drive,17, pittsburgh, pa  not_listed  ...  2014   
6696              2235 somers drive,1, pittsburgh, pa  not_listed  ...  2014   
6697             2281 somers drive,25, pittsburgh, pa  not_listed  ...  2014   
6831  320 fort duquesne boulevard,12h, pittsburgh, pa  not_listed  ...  2014   
6906   320 fort duquesne boulevard,9h, pittsburgh, pa  not_listed  ...  2014   
8002  320 fort duquesne boulevard,11m, pittsburgh, pa  not_listed  ...  2014   
8667  320 fort duquesne boulevard,21c, pittsburgh, pa  not_listed  ...  2015   
8758  320 fort duquesne boulevard,20i, pittsburgh, pa  not_listed 

In [95]:
# Add unit '#' to remaining mixed units numbers (e.g., 12e)
idx = df[~df.addr2.str.contains(r'#|suite|pitts|home|carn|apt|rear|pa|build')].index.values

In [96]:
idx

array([5461, 5561, 6562, 6563, 6696, 6697, 6831, 6906, 8002, 8667, 8758,
       9159], dtype=int64)

In [97]:
for i in idx:
    df.loc[i,'addr2'] = "#"+df.loc[i,'addr2']

In [98]:
for i in idx:
    df.loc[i,'address'] = ",".join(df.iloc[i,19:22])

In [99]:
df[~df.addr2.str.contains(r'#|suite|pitts|home|carn|apt|rear|pa|build')]

Empty DataFrame
Columns: [structure_type, parcel, neighborhood, owner, issued, work description, permit, contractor, address, work_type, ward, id, cost, year, month, source, mn, clean_ward, split_addr, addr1, addr2, addr3, addr4]
Index: []

[0 rows x 23 columns]

In [106]:
df.addr2.unique()

array([' pittsburgh', '#4634', '#12e', '#19f', '#179', '#187', '#171',
       '#9', '#17', '#105', '#142', '#117', '#129', '#165', '#1', '#25',
       '#153', '#12h', '#9h', '#1604', '#902', '#11m', '#1404', '#21c',
       '#20i', '#1506', '#9c', ' #6', ' homestead', ' #711', ' #gg',
       ' #jj', ' #19a', ' #ii', ' #hh', ' #411', ' #1006', ' #7', ' #24c',
       ' #4a', ' #707', ' carnegie', ' #u', ' #o', ' #p', ' #q', ' #r',
       ' #s', ' #t', ' #v', ' #w', ' #x', ' #y', ' #z', ' #aa', ' #24h',
       ' #14e', ' #1404', ' #165', ' #142', ' #33', ' #81', ' #57', ' #n',
       ' #d', ' #a', ' #e', ' #i', ' #g', ' #b', ' #c', ' #h', ' #l',
       ' #m', ' #j', ' #k', ' #17o', ' #906', ' #601', ' #519', ' #503',
       ' #505', ' #616', ' #15212', ' #101', ' #102', ' #103', ' #104',
       ' #201', ' #202', ' #203', ' #204', ' #301', ' #303', ' #304',
       ' #401', ' #18', ' #13', ' #414', ' #306', ' #rear', ' #512',
       ' #5', ' #2', ' #21', ' #55', ' #602', ' #406', ' #b103', '

In [107]:
df.addr3.unique()

array([' pa', ' pittsburgh', ' pa 15217', ' pa 15212', ' pa 15206',
       ' pa 15214', ' pa 15213', ' pa 15233', ' pa 15224', ' pa 15201',
       ' pa 15222', ' pa 15203', ' pa 15216', ' pa 15219', ' pa 15211',
       ' pa 15205', ' pa 15232', ' pa 15220', ' pa 15210', ' pa 15208',
       ' pa 15204', ' pa 15226', ' pa 15207', ' pa 15218', ' pa 15227',
       ' pa 15221', ' pa 15234', ' pa 15215', ' pa 15238', ' pa 15120',
       ' pa 15106', ' pa 15235', '', ' pa 15260', ' pa 15282',
       ' pa 152071776', ' pa 15323', ' pa 150201', ' pa pa', ' pa 152101',
       ' pa 15202', ' pa 1', ' 15211', ' 15207', ' 15206', ' 15216',
       ' 15234', ' 15213', ' 15224', ' 15212', ' 15227', ' 15210',
       ' 15221', ' 15208', ' 15203', ' 15219', ' 15217', ' 15233',
       ' 15235', ' 15232'], dtype=object)

In [110]:
df[df.address.str.contains(r' pa 1$')]

structure_type            parcel neighborhood               owner  \
47393     not_listed  0053b00100000000   not_listed  city of pittsburgh   

                    issued                                   work description  \
47393  2018-09-26t00:00:00  new hazardous exhaust systems/hvac systems rel...   

           permit          contractor  \
47393  18-m-00097  scalise industries   

                                       address   work_type  ...  year  \
47393  431 hamerschlag drive, pittsburgh, pa 1  not_listed  ...  2018   

           month  source  mn clean_ward  \
47393  september     API   9       14.0   

                                        split_addr                  addr1  \
47393  [431 hamerschlag drive,  pittsburgh,  pa 1]  431 hamerschlag drive   

             addr2  addr3 addr4  
47393   pittsburgh   pa 1        

[1 rows x 23 columns]

In [112]:
df.loc[47393,'address'] = '431 hamerschlag drive, pittsburgh, pa'

In [114]:
idx = df[df.address.str.contains(r'[0-9]{6,}')].index.values

In [116]:
df.loc[idx]['addr3']

20638     pa 152071776
24776     pa 152071776
24997     pa 152071776
30878        pa 150201
40861        pa 152101
Name: addr3, dtype: object

In [129]:
for i in idx:
    df.loc[i, 'addr3'] = df.loc[i, 'addr3'].split()[0] + " " + df.loc[i, 'addr3'].split()[1][:5]

In [130]:
df[df.address.str.contains(r'[0-9]{6,}')]

structure_type            parcel neighborhood                owner  \
20638     not_listed  0011r00004000001   not_listed            almono lp   
24776     not_listed  0011r00004000001   not_listed            almono lp   
24997     not_listed  0011r00004000001   not_listed            almono lp   
30878     not_listed  0049f00052000000   not_listed  jmt development inc   
40861     not_listed  0049c00266000000   not_listed      paul yanulavich   

         issued                                   work description  \
20638  6/2/2016  construction on new storage building (148ft by...   
24776       NaN  electric associated with 16-b-01161 - service ...   
24997       NaN  revised: constriction of new, one story office...   
30878  5/3/2017  (amendment to 16-m-00293) install (2) furnaces...   
40861  3/8/2018  replace existing windows at a two-story, singl...   

           permit                 contractor  \
20638  16-b-01162   franjo construction corp   
24776  16-e-01871                        NaN   
24997  16-b-01161   franjo construction corp   
30878  17-m-00711  day & night heating & air   
40861  18-b-00758     thermo twin industries   

                                             address   work_type  ...  year  \
20638  4936 gloster street, pittsburgh, pa 152071776  not_listed  ...  2016   
24776  4964 gloster street, pittsburgh, pa 152071776  not_listed  ...  2016   
24997  4964 gloster street, pittsburgh, pa 152071776  not_listed  ...  2016   
30878   4063 street johns way, pittsburgh, pa 150201  not_listed  ...  2017   
40861    4611 bancroft street, pittsburgh, pa 152101  not_listed  ...  2018   

          month  source  mn clean_ward  \
20638      june    xslx   6       15.0   
24776  november     API  11       15.0   
24997  november     API  11       15.0   
30878       may    xslx   5        9.0   
40861     march    xslx   3        9.0   

                                              split_addr  \
20638  [4936 gloster street,  pittsburgh,  pa 152071776]   
24776  [4964 gloster street,  pittsburgh,  pa 152071776]   
24997  [4964 gloster street,  pittsburgh,  pa 152071776]   
30878   [4063 street johns way,  pittsburgh,  pa 150201]   
40861    [4611 bancroft street,  pittsburgh,  pa 152101]   

                       addr1        addr2     addr3 addr4  
20638    4936 gloster street   pittsburgh  pa 15207        
24776    4964 gloster street   pittsburgh  pa 15207        
24997    4964 gloster street   pittsburgh  pa 15207        
30878  4063 street johns way   pittsburgh  pa 15020        
40861   4611 bancroft street   pittsburgh  pa 15210        

[5 rows x 23 columns]

In [131]:
for i in idx:
    df.loc[i,'address'] = ",".join(df.iloc[i,19:22])

In [152]:
df.addr3.unique()

array([' pa', ' pittsburgh', ' pa 15217', ' pa 15212', ' pa 15206',
       ' pa 15214', ' pa 15213', ' pa 15233', ' pa 15224', ' pa 15201',
       ' pa 15222', ' pa 15203', ' pa 15216', ' pa 15219', ' pa 15211',
       ' pa 15205', ' pa 15232', ' pa 15220', ' pa 15210', ' pa 15208',
       ' pa 15204', ' pa 15226', ' pa 15207', ' pa 15218', ' pa 15227',
       ' pa 15221', ' pa 15234', ' pa 15215', ' pa 15238', ' pa 15120',
       ' pa 15106', ' pa 15235', '', ' pa 15260', ' pa 15282', 'pa 15207',
       ' pa 15323', 'pa 15020', 'pa 15210', ' pa 15202', ' 15211',
       ' 15207', ' 15206', ' 15216', ' 15234', ' 15213', ' 15224',
       ' 15212', ' 15227', ' 15210', ' 15221', ' 15208', ' 15203',
       ' 15219', ' 15217', ' 15233', ' 15235', ' 15232'], dtype=object)

In [145]:
df.address = df.address.str.replace(' pa pa', ' pa')

In [147]:
df['split_addr'] = df.loc[:,'address'].str.split(',')

In [148]:
x = 0
for i in df['address'].str.split(','):
    if len(i) > x:
        x = len(i)

In [149]:
x

4

In [150]:
df['addr1'] = df.loc[:,'split_addr'].apply(lambda x: x[0] if len(x)>0 else '')

In [151]:
df['addr2'] = df.loc[:,'split_addr'].apply(lambda x: x[1] if len(x)>1 else '')
df['addr3'] = df.loc[:,'split_addr'].apply(lambda x: x[2] if len(x)>2 else '')
df['addr4'] = df.loc[:,'split_addr'].apply(lambda x: x[3] if len(x)>3 else '')

In [153]:
df.to_csv('working.csv', index=False)

In [154]:
df.addr4.unique()

array(['', ' pa 15234', ' pa 15232', ' pa 15222', ' pa 15201',
       ' pa 15213', ' pa 15210', ' pa 15211', ' pa 15219', ' pa 15206',
       ' pa 15217', ' pa 15208', ' pa', ' pa 15207', ' pa 15205',
       ' pa 15212', ' pa 15203', ' pa 15233', ' pa 15221', ' pa 15224',
       ' pa 15226'], dtype=object)

In [162]:
df

structure_type                parcel    neighborhood  \
0         not_listed      0048s00284000000      not_listed   
1         not_listed      0006r00206000000      not_listed   
2         not_listed      0006r00206000000      not_listed   
3         not_listed      0088r00175000000      not_listed   
4         not_listed      0027m00104000000      not_listed   
...              ...                   ...             ...   
70807    residential  0084-m-00316-0000-00       shadyside   
70808     commercial  0025-k-00050-0000-00  strip district   
70809     commercial  0084-b-00200-0000-00    east liberty   
70810     commercial  0083-p-00241-0000-00    east liberty   
70811    residential  0084-m-00316-0000-00       shadyside   

                             owner               issued  \
0      pfohl donald e & lucy g (w)  2012-01-30 00:00:00   
1                p7 investments lp  2012-01-04 00:00:00   
2                    dugan timothy  2012-01-04 00:00:00   
3           browns hill venture lp  2012-01-17 00:00:00   
4                 mellon institute  2012-01-19 00:00:00   
...                            ...                  ...   
70807           horen james gorman  2021-01-29t00:00:00   
70808  2614 smallman associates lp  2021-01-29t00:00:00   
70809                   alpha 4 lp  2021-01-29t00:00:00   
70810              arnold babar lp  2021-01-29t00:00:00   
70811           horen james gorman  2021-01-29t00:00:00   

                                        work description         permit  \
0      interior fitout for yoga studio\n-pittsburgh b...     11-b-03000   
1      new 3 story single family dwelling with 2 car ...     11-b-03019   
2      new 3 story single family dwelling with 2 car ...     11-b-03021   
3      tenant fitout of 2008 sq ft retail space in on...     11-b-03148   
4      interior renovations to 2200 sq ft as lab spac...     11-b-03196   
...                                                  ...            ...   
70807                                 furnace & ductwork  mp-2021-01422   
70808  new exterior mechanical equipment and interior...  mp-2020-18973   
70809  installation of multiple split system units an...  mp-2020-19379   
70810  use of 1484 sf on partial second floor as offi...  mp-2020-19381   
70811  3-story addition at rear of 3-story structure,...  mp-2021-00562   

                        contractor  \
0              nh construction llc   
1              heartland homes inc   
2              heartland homes inc   
3           gem construction, inc.   
4             mosites construction   
...                            ...   
70807  fazio heating & cooling llc   
70808                w g tomko inc   
70809          lakewood mechanical   
70810          lakewood mechanical   
70811  fazio heating & cooling llc   

                                                address  \
0                      3418 penn avenue, pittsburgh, pa   
1                 435 sweetbriar street, pittsburgh, pa   
2             437 sweetbriar street 435, pittsburgh, pa   
3               4656 browns hill road,#4634, pittsburgh   
4                     4400 fifth avenue, pittsburgh, pa   
...                                                 ...   
70807        323 denniston street, pittsburgh, pa 15206   
70808             211 26th street, pittsburgh, pa 15222   
70809            5900 penn avenue, pittsburgh, pa 15206   
70810  211 north whitfield street, pittsburgh, pa 15206   
70811        323 denniston street, pittsburgh, pa 15206   

                   work_type  ...  year    month  source  mn clean_ward  \
0                 not_listed  ...  2012  january     xls   1        6.0   
1                 not_listed  ...  2012  january     xls   1       19.0   
2                 not_listed  ...  2012  january     xls   1       19.0   
3                 not_listed  ...  2012  january     xls   1       15.0   
4                 not_listed  ...  2012  january     xls   1        4.0   
...                      ...  ...  